# OSM Oslo city bike data

The official [Oslo city bike app](https://oslobysykkel.no/en) doesn't work properly without internet connectivity. Many bysykkel station locations are still available using offline mobile apps, sush as the OSM based [maps.me](https://maps.me/en/home). However, not all of the stations are included in OSM. The purpose of this project is to look at the quality of the OSM data in this regard, and to learn how to better use python tools for processing and visualising geospatial data in a simple way

This document is written as a [Jupyter notebook](https://maps.me/en/home), you may be missing out on interactive elements if you are reading this on eg. GitHub.

# Prerequisites

python (tested in python 3.5) packages used:

- pandas
- geopandas
- shapely
- folium
- numpy
- matplotlib
- json
- requests
- descartes

# Todos

- figure out how to use folium cluster labels [maybe based on this](https://ocefpaf.github.io/python4oceanographers/blog/2015/12/14/geopandas_folium/)
- folium popus with geopandas data

# Authoritative data source 

Bysykkel station data can be extracted extracted from the map at https://oslobysykkel.no/en/map. The map is populated with json formatted data from https://oslobysykkel.no/api/internal/stations.

Read the json from the api:

In [1]:
import requests
import json
import geopandas as gpd
oslo_json = requests.get("https://oslobysykkel.no/api/internal/stations")
oslo_json = oslo_json.content.decode(oslo_json.apparent_encoding)
oslo_json = json.loads(oslo_json)
oslo_json['stations'][:3]

[{'center': {'latitude': 59.929838, 'longitude': 10.711223},
  'id': 178,
  'ready': True,
  'subtitle': 'langs Fridtjof Nansens vei',
  'title': 'Colosseum Kino'},
 {'center': {'latitude': 59.939228, 'longitude': 10.75917},
  'id': 158,
  'ready': True,
  'subtitle': 'rett over busstoppet',
  'title': 'Bentsebrugata'},
 {'center': {'latitude': 59.939238, 'longitude': 10.774279},
  'id': 159,
  'ready': True,
  'subtitle': 'Mellom Åsengata og Nordkappgata',
  'title': 'Hans Nielsen Hauges plass'}]

Convert the dataset into a geojson feature collecion in order to do GIS stuff  with it more easily. See http://geojson.org/geojson-spec.html for more details about the structure of geojson objects.

Convert the data into a geojson `FeatureCollection`

In [2]:
import geojson

oslo_stations_features = []*0

for station in oslo_json['stations']:
    #print(json.dumps(station, indent=2))
    point = geojson.Point([station['center']['longitude'], station['center']['latitude']])
    station_properties = dict(station).pop('center')
    feature = geojson.Feature(geometry = point, properties = station_properties)
    oslo_stations_features.append(feature)
oslo_featurecollection = geojson.FeatureCollection(oslo_stations_features)

The snippet below saves the newly converted geojson data into a file. This way it will be easy to ingest into other tools. `geopandas.read_file()` supports geojson files.
If there is a way to read geojson from a string into a geopandas.GeoDataFrame, it doesn't seem to be very well documented

In [3]:
geojson_string = geojson.dumps(oslo_featurecollection, indent = 2, sort_keys = True)
with open("oslo_bysykkel_station_feature_collection.geojson", 'w') as f:
    f.write(geojson_string)

Visualize the feature collection with leaflet.js using folium:

In [4]:
import folium
import geopandas as gpd

oslo_stations_d = gpd.read_file("oslo_bysykkel_station_feature_collection.geojson")

mapa = folium.Map(tiles='cartodbpositron')
mapa.fit_bounds([
    [oslo_stations_d.bounds['miny'].min(),oslo_stations_d.bounds['minx'].min()],
    [oslo_stations_d.bounds['maxy'].max(),oslo_stations_d.bounds['maxx'].max()]
])

points = folium.features.GeoJson(geojson.dumps(oslo_featurecollection))
mapa.add_children(points)
mapa


# OSM data

In [5]:
import requests
import json
osm_s = requests.get("https://gist.githubusercontent.com/jarmokivekas/b9d1cdfd006d3df26b871de28bebefe4/raw/450d1ab6f98de9307703583ea2655f29f688d09b/overpass.geojson")
osm_s = osm_s.content.decode(osm_s.apparent_encoding)
osm_s = json.loads(osm_s)

bysykkels = []
for feature in osm_s['features']:
    if feature['properties']['network'] == 'Bysykkel' :
        bysykkels.append(feature) 


In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt
overpass_d = gpd.read_file("./overpass.geojson")
overpass_d = overpass_d[overpass_d['network'] == "Bysykkel"]

# print a few exmple points 
overpass_d.head(3)

,@id,addr:city,addr:country,addr:housenumber,addr:street,alt_name,amenity,capacity,description,geometry,name,network,note,operator,ref,source,wheelchair
0,node/408455943,None,None,None,None,None,bicycle_rental,12,33-Bygdøy v/Folkemuseet,POINT (10.6868353 59.9076592),None,Bysykkel,None,ClearChannel,33,survey,None
1,node/408457759,None,None,None,None,None,bicycle_rental,30,51-Havnegata S (Østbanehallen) v/Hotell Opera,POINT (10.7521008 59.910089),None,Bysykkel,None,ClearChannel,51,survey,None
2,node/408457760,None,None,None,None,None,bicycle_rental,30,50-Havnegata N (Østbanehallen),POINT (10.7517602 59.910162),None,Bysykkel,None,ClearChannel,50,survey,None


### Leflet.js visualization using folium

In [7]:
import folium

mapa = folium.Map(tiles='cartodbpositron')
mapa.fit_bounds([
    [overpass_d.bounds['miny'].min(),overpass_d.bounds['minx'].min()],
    [overpass_d.bounds['maxy'].max(),overpass_d.bounds['maxx'].max()]
])

overpass_geojson = overpass_d.to_json()
points = folium.features.GeoJson(overpass_geojson)
mapa.add_children(points)
mapa

One way of figuring out the bounds of a set of point:

In [8]:
overpass_d.bounds['miny'].min();
overpass_d.bounds['minx'].min();
overpass_d.bounds['maxy'].max();
overpass_d.bounds['maxx'].max();



pandas has a `DataFrame.mean()` for calculating the mean value of a data column. However, due to the way points are encoded as `shaplely.geometry.Point` I haven't figured out how to use `DataFrame.mean()` on location data.

In [9]:
x_cumulative_sum = 0
y_cumulative_sum = 0
for point in overpass_d['geometry']:
    x_cumulative_sum += point.x
    y_cumulative_sum += point.y

x_cumulative_sum / len(overpass_d['geometry']);
y_cumulative_sum / len(overpass_d['geometry']);

# Actually comparing the two datasets

Process:

- find equivalent point in the two datasets
    - dilate points in both dataset using buffering
    - check for overlaps in the buffered datasets
    - merge datasets by using the centroid of the union of the overlapping buffers


Load the datasets, and make sure they have the same projetions:

In [10]:
import geopandas as gpd

overpass_d = gpd.read_file("overpass.geojson")
bysykkel_d = gpd.read_file("oslo_bysykkel_station_feature_collection.geojson")
datasets = [overpass_d, bysykkel_d]
print(overpass_d.crs)
print(bysykkel_d.crs)

{'init': 'epsg:4326'}
{'init': 'epsg:4326'}


Making buffers is likely going to be more intuitive if the coordinates are converted to easting/northin inseatd of lat/lon pairs. The projection below makes the coordinate units to be in meters. This way a `myGeoDataFrame.buffer(10)` creates a 10 meter buffer around the feature.



In [11]:
utm_zones_north_proj4 = "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

for idx, dataset in enumerate(datasets):
    datasets[idx] = dataset.to_crs({'proj': utm_zones_north_proj4})
    print(overpass_d.crs)
    

{'init': 'epsg:4326'}
{'init': 'epsg:4326'}


** somewhere around here stuff start to get wierd** 

In [12]:
#%matplotlib inline

import matplotlib.pyplot as plt

for idx, dataset in enumerate(datasets):
    plt.figure()
    datasets[idx]['geometry'] = dataset.buffer(10+2*idx)
    datasets[idx].plot()
plt.show()

In [13]:
over = datasets[0]['geometry'].unary_union
bysy = datasets[1]['geometry'].unary_union


a = gpd.GeoDataFrame()
a['geometry'] = over.intersection(bysy)
a
a.plot()

In [14]:
import shapely
import geojson
import folium
geometry = shapely.geometry.geo.mapping(over.intersection(bysy))
geometry
multi_poly = geojson.MultiPolygon(geometry['coordinates'])

In [15]:
multi_poly_feature = geojson.Feature(geometry=multi_poly, id=1)


In [16]:
multi_poly_feature_geojson = geojson.dumps(geojson.FeatureCollection(multi_poly_feature))

In [17]:
multis = folium.features.GeoJson(multi_poly_feature_geojson)
mapb = folium.Map(tiles="cartodbpositron")
mapb.add_child(multis)
mapb

AttributeError: 'str' object has no attribute 'setdefault'

In [18]:
mapb = folium.Map()
mapb.add_children(geojson.FeatureCollection(multi_poly_feature))

AttributeError: get_name

In [19]:
datasets[1]

,geometry,latitude,longitude
0,"POLYGON ((260399.1365549927 6651363.843600913,...",59.929838,10.711223
1,"POLYGON ((263142.4862022958 6652235.427037643,...",59.939228,10.759170
2,"POLYGON ((263985.6978033626 6652182.524679575,...",59.939238,10.774279
3,"POLYGON ((261619.2332675317 6650249.28955927, ...",59.920565,10.734274
4,"POLYGON ((264007.3803106746 6649547.068080164,...",59.915650,10.777671
5,"POLYGON ((263145.8891825089 6649598.852605552,...",59.915620,10.762248
6,"POLYGON ((262145.7081380723 6650099.763267561,...",59.919530,10.743836
7,"POLYGON ((261604.778206668 6651254.972138453, ...",59.929562,10.732858
8,"POLYGON ((261526.784577045 6652468.126846794, ...",59.940380,10.730068
9,"POLYGON ((262517.4921555058 6649035.602742099,...",59.910215,10.751687


# Difference (points - buffer)



In [20]:
#%matplotlib inline
import geopandas as gpd
import matplotlib.pyplot as plt
overpass_d = gpd.read_file("overpass.geojson")
bysykkel_d = gpd.read_file("oslo_bysykkel_station_feature_collection.geojson")
datasets = [overpass_d, bysykkel_d]

utm_zones_north_proj4 = "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
overpass_d = overpass_d.to_crs({'proj': utm_zones_north_proj4})
bysykkel_d = bysykkel_d.to_crs({'proj': utm_zones_north_proj4})
overpass_d = overpass_d.buffer(100)

plt.figure()
difference = bysykkel_d['geometry'] - overpass_d
difference.plot()
plt.show()

In [21]:
print(type(overpass_d))
print(type(bysykkel_d))

<class 'geopandas.geoseries.GeoSeries'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [22]:
difference.count()

150

In [23]:
plt.figure()
bysykkel_d['geometry'].plot()
overpass_d.plot()
plt.show()

In [63]:
overpass_d = gpd.read_file("overpass.geojson")
bysykkel_d = gpd.read_file("oslo_bysykkel_station_feature_collection.geojson")
datasets = [overpass_d, bysykkel_d]

utm_zones_north_proj4 = "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
overpass_d = overpass_d.to_crs({'proj': utm_zones_north_proj4})
bysykkel_d = bysykkel_d.to_crs({'proj': utm_zones_north_proj4})

bysykkel_d['geometry'] = bysykkel_d['geometry'].buffer(1)
overpass_d = overpass_d.buffer(50)

plt.figure()
difference_d = bysykkel_d
difference_d['geometry'] = bysykkel_d['geometry'] - overpass_d.unary_union
plt.title("bysykkel - overpass unary union" + str(difference_d.count()))
difference_d.plot()
plt.show()

In [64]:
difference_d[difference_d['geometry'] != None]


,geometry,latitude,longitude
0,(),59.929838,10.711223
1,"POLYGON ((263131.4862022958 6652235.427037643,...",59.939228,10.759170
2,"POLYGON ((263974.6978033626 6652182.524679575,...",59.939238,10.774279
3,(),59.920565,10.734274
4,(),59.915650,10.777671
5,(),59.915620,10.762248
6,(),59.919530,10.743836
7,"POLYGON ((261593.778206668 6651254.972138453, ...",59.929562,10.732858
8,"POLYGON ((261515.784577045 6652468.126846794, ...",59.940380,10.730068
9,(),59.910215,10.751687


In [65]:
difference_points_d = difference_d.centroid
difference_points_d

0                                      POINT EMPTY
1      POINT (263130.4862022959 6652235.427037644)
2      POINT (263973.6978033626 6652182.524679575)
3                                      POINT EMPTY
4                                      POINT EMPTY
5                                      POINT EMPTY
6                                      POINT EMPTY
7      POINT (261592.7782066679 6651254.972138453)
8      POINT (261514.7845770451 6652468.126846797)
9                                      POINT EMPTY
10                                     POINT EMPTY
11                                     POINT EMPTY
12     POINT (262911.8913867857 6649944.759849366)
13     POINT (263111.0002430443 6649252.351113127)
14     POINT (263859.5541560763 6649972.164565776)
15     POINT (261095.8815199825 6650747.354086609)
16                                     POINT EMPTY
17                                     POINT EMPTY
18     POINT (263221.1255425007 6652734.593767963)
19                             